In [ ]:
import zipfile
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import IncrementalPCA
import time


In [ ]:


# Step 2: Load and preprocess images in batches
def load_images_in_batches(image_paths, batch_size=16, target_size=(64, 64)):
    for start in range(0, len(image_paths), batch_size):
        end = min(start + batch_size, len(image_paths))
        images = []
        for image_path in image_paths[start:end]:
            img = cv2.imread(image_path)
            if img is not None:
                img_resized = cv2.resize(img, target_size)
                img_normalized = img_resized / 255.0
                img_flattened = img_normalized.flatten()
                images.append(img_flattened)
        yield np.array(images)



In [ ]:

# Step 3: Extract labels
def extract_labels(image_paths):
    labels = []
    for path in tqdm(image_paths, desc="Extracting labels"):
        filename = os.path.basename(path)
        if filename.startswith('cat'):
            labels.append(0)
        elif filename.startswith('dog'):
            labels.append(1)
    return np.array(labels)

train_image_paths = [os.path.join(train_dir, 'train', img) for img in train_images]
y_train = extract_labels(train_image_paths)


In [ ]:

# Step 4: Split into train and validation
train_paths, val_paths, y_train_split, y_val = train_test_split(
    train_image_paths, y_train, test_size=0.2, random_state=42
)

# Step 5: Incremental PCA
print("Applying IncrementalPCA to reduce feature dimensions...")
batch_size = 16
ipca = IncrementalPCA(n_components=16)

for batch in tqdm(load_images_in_batches(train_paths, batch_size), desc="Fitting IncrementalPCA"):
    ipca.partial_fit(batch)

X_train_pca = np.vstack([
    ipca.transform(batch) for batch in tqdm(load_images_in_batches(train_paths, batch_size), desc="Transforming train data")
])
X_val_pca = np.vstack([
    ipca.transform(batch) for batch in tqdm(load_images_in_batches(val_paths, batch_size), desc="Transforming validation data")
])


In [ ]:

# Step 6: Train the SVM
print("Training the SVM classifier...")
start_time = time.time()
svm_model = SVC(kernel='rbf', C=100.0, random_state=42)
svm_model.fit(X_train_pca, y_train_split)
training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

# Step 7: Evaluate
val_accuracy = svm_model.score(X_val_pca, y_val)
print(f"Validation Accuracy: {val_accuracy:.2f}")


Training the SVM classifier...
